# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
y.values

array([0, 0, 0, ..., 1, 0, 1])

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=['credit_ranking'])

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
n_features = X_train.shape[1]

# Review the number of features
n_features

11

In [10]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer_a = 10

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer_b = 5


# Define the number of neurons in the output layer
n_output = 1

In [11]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(hidden_nodes_layer_a, input_dim=n_features, activation='relu'))

# Add the second hidden layer
model.add(Dense(hidden_nodes_layer_b, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(n_output, activation='sigmoid'))

In [12]:
# Display the Sequential model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                120       
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 181 (724.00 Byte)
Trainable params: 181 (724.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Fit the model using 50 epochs and the training data
epochs_fitted = model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
35/35 [==============================] - 3s 8ms/step - loss: 0.7231 - accuracy: 0.3950
Epoch 2/50
35/35 [==============================] - 0s 11ms/step - loss: 0.7018 - accuracy: 0.4710
Epoch 3/50
35/35 [==============================] - 0s 9ms/step - loss: 0.6876 - accuracy: 0.5567
Epoch 4/50
35/35 [==============================] - 0s 13ms/step - loss: 0.6752 - accuracy: 0.6363
Epoch 5/50
35/35 [==============================] - 0s 12ms/step - loss: 0.6626 - accuracy: 0.6872
Epoch 6/50
35/35 [==============================] - 0s 12ms/step - loss: 0.6485 - accuracy: 0.6997
Epoch 7/50
35/35 [==============================] - 0s 7ms/step - loss: 0.6324 - accuracy: 0.7194
Epoch 8/50
35/35 [==============================] - 0s 8ms/step - loss: 0.6180 - accuracy: 0.7230
Epoch 9/50
35/35 [==============================] - 0s 10ms/step - loss: 0.6051 - accuracy: 0.7310
Epoch 10/50
35/35 [==============================] - 0s 12ms/step - loss: 0.5928 - accuracy: 0.7462
Epoch 11/50
3

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print("Loss:", loss)
print("Accuracy:", accuracy)

15/15 [==============================] - 0s 2ms/step - loss: 0.5255 - accuracy: 0.7667
Loss: 0.5254788994789124
Accuracy: 0.7666666507720947


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
# Set the model's file path
model_file_path = "student_loans.keras"

# Export your model to a keras file
model.save(model_file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [22]:
# Set the model's file path


# Load the model to a new object
from tensorflow.keras.models import load_model
reloaded_model = load_model("student_loans.keras")

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [30]:
# Make predictions with the test data
predictions = reloaded_model.predict(X_test_scaled)

# Display a sample of the predictions
print(predictions[10])

15/15 [==============================] - 0s 6ms/step
[0.9122397]


In [36]:
# Save the predictions to a DataFrame and round the predictions to binary results
import numpy as np
predictions_df = pd.DataFrame({'Predicted_Credit_Ranking': np.round(predictions.flatten()).astype(int)})

### Step 4: Display a classification report with the y test data and predictions

In [37]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df['Predicted_Credit_Ranking']))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76       230
           1       0.78      0.78      0.78       250

    accuracy                           0.77       480
   macro avg       0.77      0.77      0.77       480
weighted avg       0.77      0.77      0.77       480



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

  1.A.) This data includes details such as age, gender, location, ethnicity, field of study, academic performance and household income. Analyzing the datasets can help the recommendation algorithm to generate personalized loan options tailored to each student.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

  2.A.) By leveraging the rich features of both students and loan options, content-based filtering can generate personalized recommendations tailored to each student's unique profile and needs.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

  3.A.) The dataset used to train and evaluate the recommendation system contains highly sensitive information, including students' demographic details, academic performance, and financial history. Security of this data is essential as unauthorized access to student data can lead to severe consequences.

  3.B.)  Lenders will revise their lending policies, and terms and conditions in response to regulations, trends, or a new business strategy. Recommendation systems need to stay informed about such policy changes and incorporate them into their recommendation algorithms to provide relevant recommendations.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

